In [9]:
import sys,os
%load_ext autoreload
%autoreload 2
print(sys.path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.5/dist-packages/IPython/extensions', '/root/.ipython']


In [10]:
from pretrain import span_mask_utils
from pretrain import pretrain_helpers

In [13]:
from bunch import Bunch
import numpy as np
import tensorflow as tf


FLAGS = Bunch({})
FLAGS.min_tok = 3
FLAGS.max_tok = 10
FLAGS.sep_id = 102
FLAGS.pad_id = 0
FLAGS.cls_id = 101
FLAGS.mask_id = 103
FLAGS.batch_size = 1000
FLAGS.leak_ratio = 0.1
FLAGS.rand_ratio = 0.1
FLAGS.vocab_size = 21128
FLAGS.mask_prob = 0.15
FLAGS.sample_strategy = 'token_span'
FLAGS.confusion_matrix = None
FLAGS.confusion_mask_matrix = None
FLAGS.prepare_text_infilling = False
FLAGS.initial_ratio = 0.1
FLAGS.final_ratio = 0.3
FLAGS.num_train_steps = 1000

name_to_features = {
            "input_ori_ids":
                tf.FixedLenFeature([512], tf.int64)
}
record_spec = {
				"input_ori_ids":
						tf.FixedLenFeature([512], tf.int64),
# 				"input_mask":
# 						tf.FixedLenFeature([512], tf.int64),
				"segment_ids":
						tf.FixedLenFeature([512], tf.int64),
	}

def _decode_record(record, name_to_features, **kargs):
    example = tf.parse_single_example(record, name_to_features)
    return example

def train_input_fn(input_file, _parse_fn, name_to_features,
                   params,
                   num_predict=78, seq_len=512,
                   use_bfloat16=False,
                   truncate_seq=False,
                   stride=1,
                    **kargs):

    dataset = tf.data.TFRecordDataset(input_file, buffer_size=params.get("buffer_size", 100))
#     dataset = dataset.shuffle(1024)
#     dataset = dataset.map(lambda x:_parse_fn(x, record_spec))
    dataset = dataset.map(lambda x:span_mask_utils._decode_record(FLAGS, x, num_predict,
                                    seq_len, 
                                    use_bfloat16=use_bfloat16, 
                                    truncate_seq=truncate_seq, 
                                    stride=stride))
    dataset = dataset.batch(params.get("batch_size", 1))
    dataset = dataset.repeat(params.get("epoch", 1))
    iterator = dataset.make_one_shot_iterator()
    features = iterator.get_next()
    return features

output = ['/data/xuht/chinese_sub_task_0.tfrecord']
input_fn = train_input_fn(output[0], _decode_record, name_to_features, params=FLAGS)

sess = tf.Session()

init_op = tf.group(
            tf.local_variables_initializer())
sess.run(init_op)
cout = 0
while True:
    features = sess.run(input_fn)
    cout += 1
    break

INFO:tensorflow:Online sample with strategy: `token_span`.
==tf version== 1.12.0
==tf version== 1.12.0
INFO:tensorflow:masked_input: Tensor("Cast_23:0", shape=(512,), dtype=int32)
INFO:tensorflow:segment_ids: Tensor("Cast_24:0", shape=(512,), dtype=int32)
INFO:tensorflow:masked_lm_positions: Tensor("Cast_25:0", shape=(78,), dtype=int32)
INFO:tensorflow:masked_mask: Tensor("sub_6:0", shape=(512,), dtype=float32)
INFO:tensorflow:masked_lm_weights: Tensor("concat_1:0", shape=(78,), dtype=float32)
INFO:tensorflow:target_mapping: Tensor("Reshape_3:0", shape=(78, 512), dtype=float32)
INFO:tensorflow:target_mask: Tensor("concat_1:0", shape=(78,), dtype=float32)
INFO:tensorflow:target: Tensor("Cast_26:0", shape=(78,), dtype=int32)
INFO:tensorflow:masked_lm_ids: Tensor("Cast_27:0", shape=(78,), dtype=int32)
INFO:tensorflow:is_target: Tensor("Cast_28:0", shape=(512,), dtype=int32)
INFO:tensorflow:origin_input: Tensor("Cast_29:0", shape=(512,), dtype=int32)
INFO:tensorflow:pad_mask: Tensor("sub_7

In [14]:
unmasked_input_ids, _ = pretrain_helpers.scatter_update(
  tf.constant(features["masked_input"]), 
  tf.constant(features["masked_lm_ids"]), 
  tf.constant(features["masked_lm_positions"]))
resp = sess.run(unmasked_input_ids)

In [16]:
resp[0]-features['origin_input'][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
import six
def get_shape_list(tensor, expected_rank=None, name=None):
  """Returns a list of the shape of tensor, preferring static dimensions.

  Args:
    tensor: A tf.Tensor object to find the shape of.
    expected_rank: (optional) int. The expected rank of `tensor`. If this is
      specified and the `tensor` has a different rank, and exception will be
      thrown.
    name: Optional name of the tensor for the error message.

  Returns:
    A list of dimensions of the shape of tensor. All static dimensions will
    be returned as python integers, and dynamic dimensions will be returned
    as tf.Tensor scalars.
  """
  if name is None:
    name = tensor.name

  if expected_rank is not None:
    assert_rank(tensor, expected_rank, name)

  shape = tensor.shape.as_list()

  non_static_indexes = []
  for (index, dim) in enumerate(shape):
    if dim is None:
      non_static_indexes.append(index)

  if not non_static_indexes:
    return shape

  dyn_shape = tf.shape(tensor)
  for index in non_static_indexes:
    shape[index] = dyn_shape[index]
  return shape


def reshape_to_matrix(input_tensor):
  """Reshapes a >= rank 2 tensor to a rank 2 tensor (i.e., a matrix)."""
  ndims = input_tensor.shape.ndims
  if ndims < 2:
    raise ValueError("Input tensor must have at least rank 2. Shape = %s" %
                     (input_tensor.shape))
  if ndims == 2:
    return input_tensor

  width = input_tensor.shape[-1]
  output_tensor = tf.reshape(input_tensor, [-1, width])
  return output_tensor


def reshape_from_matrix(output_tensor, orig_shape_list):
  """Reshapes a rank 2 tensor back to its original rank >= 2 tensor."""
  if len(orig_shape_list) == 2:
    return output_tensor

  output_shape = get_shape_list(output_tensor)

  orig_dims = orig_shape_list[0:-1]
  width = output_shape[-1]

  return tf.reshape(output_tensor, orig_dims + [width])


def assert_rank(tensor, expected_rank, name=None):
  """Raises an exception if the tensor rank is not of the expected rank.

  Args:
    tensor: A tf.Tensor to check the rank of.
    expected_rank: Python integer or list of integers, expected rank.
    name: Optional name of the tensor for the error message.

  Raises:
    ValueError: If the expected shape doesn't match the actual shape.
  """
  if name is None:
    name = tensor.name

  expected_rank_dict = {}
  if isinstance(expected_rank, six.integer_types):
    expected_rank_dict[expected_rank] = True
  else:
    for x in expected_rank:
      expected_rank_dict[x] = True

  actual_rank = tensor.shape.ndims
  if actual_rank not in expected_rank_dict:
    scope_name = tf.get_variable_scope().name
    raise ValueError(
        "For the tensor `%s` in scope `%s`, the actual rank "
        "`%d` (shape = %s) is not equal to the expected rank `%s`" %
        (name, scope_name, actual_rank, str(tensor.shape), str(expected_rank)))


In [11]:
import tensorflow as tf

def iterative_inv(mat, n_iter=6):

  """
  https://downloads.hindawi.com/journals/aaa/2014/563787.pdf
  A New Iterative Method for Finding Approximate Inverses of
  Complex Matrices
  """

  mat_shape = get_shape_list(mat, expected_rank=[2,3,4])
  I = tf.cast(tf.eye(mat_shape[-1]), dtype=tf.float32)
  K = tf.identity(mat) 
  # [B, N, n-landmarks, n-landmarks]
  V = 1 / (tf.reduce_max(tf.reduce_sum(tf.abs(K), axis=-2)) * tf.reduce_max(tf.reduce_sum(tf.abs(K), axis=-1))) * tf.transpose(K, [0,1,3,2])

  for _ in range(n_iter):
      KV = tf.matmul(K, V)
      V = tf.matmul(0.25 * V, 13 * I - tf.matmul(KV, 15 * I - tf.matmul(KV, 7 * I - KV)))
  # [B, N, n-landmarks, n-landmarks]
  # V = tf.stop_gradient(V)
  return V

In [12]:
import numpy as np

In [91]:
mat = np.random.random((2, 4, 32,32)).astype(np.float32)

In [56]:
mat = np.random.normal(0,1,(2,4,3,3)).astype(np.float32)

In [100]:
p_inv = iterative_inv(tf.constant(mat), 20)
sess = tf.Session()
resp_tf = sess.run(p_inv)

In [101]:
resp_tf[0,0]

array([[  2.4286504 ,  -1.8182744 ,  -1.4012866 , ..., -11.029692  ,
         -1.934946  ,   1.7226888 ],
       [  1.8924295 ,  -2.8874543 ,  -0.91047823, ..., -11.661704  ,
         -2.3886647 ,   2.4778032 ],
       [ -1.934705  ,   2.9703434 ,   1.1450763 , ...,  13.489203  ,
          2.979328  ,  -1.4722278 ],
       ...,
       [  1.5465115 ,  -2.7143054 ,  -0.23415425, ...,  -6.2651434 ,
         -1.6216046 ,   1.270768  ],
       [  0.21798922,  -1.0885704 ,  -0.8010062 , ...,  -8.297365  ,
         -1.8994988 ,   0.3882413 ],
       [  0.13643757,   0.20728293,  -0.34668654, ...,   1.6374046 ,
          0.17163077,   0.8526275 ]], dtype=float32)

In [102]:
np.linalg.pinv(mat[0,0])

array([[  2.4286497 ,  -1.8183264 ,  -1.4012816 , ..., -11.029798  ,
         -1.9349804 ,   1.7226788 ],
       [  1.892427  ,  -2.8875146 ,  -0.910474  , ..., -11.661825  ,
         -2.3887038 ,   2.477794  ],
       [ -1.9347032 ,   2.9704075 ,   1.1450713 , ...,  13.489328  ,
          2.9793696 ,  -1.472217  ],
       ...,
       [  1.5465101 ,  -2.7143383 ,  -0.23415156, ...,  -6.265218  ,
         -1.6216245 ,   1.2707617 ],
       [  0.2179893 ,  -1.0886066 ,  -0.80100286, ...,  -8.297422  ,
         -1.8995227 ,   0.38823432],
       [  0.13643639,   0.20728822,  -0.3466876 , ...,   1.6374053 ,
          0.17163214,   0.85262936]], dtype=float32)

In [70]:
np.linalg.matrix_rank(mat[0,0])

3